# Mount Google Drive

In [5]:
# from google.colab import drive
# drive.mount("/content/drive/")

In [6]:
# !pip3 install fse
# !pip3 install pytorch-nlp

# conda install -c anaconda nltk
#  conda install -c anaconda pandas
# conda create -n sklearn-env -c conda-forge scikit-learn
# conda activate sklearn-env
# conda install pytorch torchvision torchaudio -c pytorch

# Import Libraries

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import re
import string
import collections
import fse
import random

import random
import math
import time

import itertools
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchnlp.metrics import get_moses_multi_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1)

import nltk
nltk.download('punkt')

import nltk
from nltk.util import ngrams
nltk.download('punkt')
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Subset
from sklearn.model_selection import train_test_split

# Any results you write to the current directory are saved as output.

/Users/nurrizkyimani/.conda/envs/workspace/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nurrizkyimani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nurrizkyimani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Importing Dataset GDrive

In [8]:
# # import dataset from google drive
# path = "/content/drive/MyDrive/nlp_research/2022_thesis_styletransfer/annotated/combined/csv/"
# all_agree_path = path + "all_agree.csv"
# all_agree_pd = pd.read_csv(all_agree_path)
# df_clickbait = all_agree_pd[all_agree_pd['label'] == "clickbait"]
# df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
# df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)
#
# df_both_copy = df_both.copy()
# df_both_copy['label'] = df_both_copy['label'].astype('category')
# df_both_copy.head(-1)

# Importing dataset

In [9]:
# Import the csv
all_agree_path = "2022_thesis_styletransfer/annotated/combined/csv/all_agree.csv"
all_agree_pd = pd.read_csv(all_agree_path)
df_clickbait = all_agree_pd[all_agree_pd['label'] == "clickbait"]
df_nonclickbait = all_agree_pd[all_agree_pd['label'] == "non-clickbait"]
df_both = pd.concat((df_clickbait, df_nonclickbait), ignore_index=True)

df_both_copy = df_both.copy()
df_both_copy['label'] = df_both_copy['label'].astype('category')
df_both_copy.head(-1)

,title,label,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,1
3,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,1
4,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",clickbait,1
...,...,...,...
8607,"Smart SIM Diluncurkan, Wakapolri Harap Bisa Me...",non-clickbait,0
8608,"Wamena Papua Kembali Membara, Kantor Bupati Di...",non-clickbait,0
8609,BMKG Angkat Bicara Soal Langit Merah Jambi,non-clickbait,0
8610,Operasional Bandara Wamena Dihentikan Akibat R...,non-clickbait,0


In [10]:
df_both_label = df_both_copy.drop('label_score', axis=1)
df_both = df_both_copy.drop('label', axis=1)
df_both.head(-5)

,title,label_score
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,1
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",1
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,1
3,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,1
4,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",1
...,...,...
8603,Komnas HAM Minta Aparat Keamanan Tak Gunakan K...,0
8604,Mahasiswa Minta TNI Turun Bersama Kawal Aksi D...,0
8605,Polisi Pulangkan 56 Mahasiswa Setelah Sempat D...,0
8606,KPK Tepis Tudingan Moeldoko Soal Hambat Investasi,0


In [11]:
len(df_both)

8613

In [12]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
df_both_copy['label_encoding'] = labelencoder.fit_transform(df_both_copy['label'])
df_encode_clean = df_both_copy.copy()
df_encode_clean = df_both = df_both_copy.drop(['label_score'], axis=1)
df_encode_clean.head()

,title,label,label_encoding
0,Viral! Driver Ojol di Bekasi Antar Pesanan Mak...,clickbait,0
1,"Ada Motor Nyangkut di Atas Bambu di Sleman, Ko...",clickbait,0
2,Pesan Gamblang Poyuono Menolak Revisi UU KPK,clickbait,0
3,Kocak! Maling di Rumah Mewah Jakut Terekam CCT...,clickbait,0
4,"Viral Video Diduga Baku Tembak di Sleman, Ini ...",clickbait,0


# Preprocessing : Lower Casing

In [16]:
# preprocessing : lower casing
df_encode_clean['title'] = df_encode_clean['title'].str.lower()
df_encode_clean.title

0       viral! driver ojol di bekasi antar pesanan mak...
1       ada motor nyangkut di atas bambu di sleman, ko...
2            pesan gamblang poyuono menolak revisi uu kpk
3       kocak! maling di rumah mewah jakut terekam cct...
4       viral video diduga baku tembak di sleman, ini ...
                              ...                        
8608    wamena papua kembali membara, kantor bupati di...
8609           bmkg angkat bicara soal langit merah jambi
8610    operasional bandara wamena dihentikan akibat r...
8611     asap karhutla riau mulai merambah ke nias, bm...
8612    tolak ruu pertanahan, ribuan petani siap gelar...
Name: title, Length: 8613, dtype: object

Convert the titl into lower case for all label

In [17]:
# Add space between punctuation
df_encode_clean_t =  df_encode_clean.copy()
df_encode_clean_t['title'] = df_encode_clean_t['title'].apply(lambda y: " ".join((re.sub(r'([!/?/./,/%/\'/\"/-/:])', lambda x: ' ' + x.group()+' ' , y)).split()) , 1)
df_encode_clean_t

,title,label,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,clickbait,0
1,"ada motor nyangkut di atas bambu di sleman , k...",clickbait,0
2,pesan gamblang poyuono menolak revisi uu kpk,clickbait,0
3,kocak ! maling di rumah mewah jakut terekam cc...,clickbait,0
4,"viral video diduga baku tembak di sleman , ini...",clickbait,0
...,...,...,...
8608,"wamena papua kembali membara , kantor bupati d...",non-clickbait,1
8609,bmkg angkat bicara soal langit merah jambi,non-clickbait,1
8610,operasional bandara wamena dihentikan akibat r...,non-clickbait,1
8611,"asap karhutla riau mulai merambah ke nias , bm...",non-clickbait,1


In [18]:
# separate between clickbait clean and nonclickbait clean
df_clickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "clickbait"]
df_nonclickbait_clean = df_encode_clean_t[df_encode_clean_t['label'] == "non-clickbait"]

In [19]:
# create a list form the combination of df_clickbait and nonclickbait

d_both = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both = d_both.drop(['label', 'label_encoding'], axis=1)
d_both = d_both.values.tolist()
d_both = list(itertools.chain.from_iterable(d_both))

In [20]:
d_both_clean = pd.concat((df_clickbait_clean, df_nonclickbait_clean ), ignore_index=True)
d_both_clean = d_both_clean.drop(['label'], axis=1)
d_both_clean

,title,label_encoding
0,viral ! driver ojol di bekasi antar pesanan ma...,0
1,"ada motor nyangkut di atas bambu di sleman , k...",0
2,pesan gamblang poyuono menolak revisi uu kpk,0
3,kocak ! maling di rumah mewah jakut terekam cc...,0
4,"viral video diduga baku tembak di sleman , ini...",0
...,...,...
8608,"wamena papua kembali membara , kantor bupati d...",1
8609,bmkg angkat bicara soal langit merah jambi,1
8610,operasional bandara wamena dihentikan akibat r...,1
8611,"asap karhutla riau mulai merambah ke nias , bm...",1


In [21]:
d_both

['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda',
 'ada motor nyangkut di atas bambu di sleman , kok bisa ?',
 'pesan gamblang poyuono menolak revisi uu kpk',
 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur',
 'viral video diduga baku tembak di sleman , ini kata polisi',
 'waspada ! ada penipuan catut pertamina , korbannya rugi puluhan juta',
 'jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ini rinciannya',
 'sering quality time bersama keluarga ? ternyata ini 3 manfaatnya !',
 'menteri jokowi 55 % profesional , ini kandidatnya ?',
 "pria misterius berjubah putih viral di sumut , ingatkan ' tuhan murka '",
 'video porno di sumedang disebar pemeran pria , apa motifnya ?',
 'terungkap ! video porno di sumedang diperankan pasangan selingkuh',
 'kendaraan listrik bebas dari aturan ganjil-genap , ini alasannya',
 'menanti " kebijakan gila " jokowi',
 'akhirnya ! pemkot pekanbaru liburkan sd dan smp karena kabut asap',
 '2 ribu hekt

In [22]:
# create list of clickbait and nonclickbait headline

df_clickbait_t = df_clickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
clickbait_l_c = list(itertools.chain.from_iterable(df_clickbait_t))

df_nonclickbait_t = df_nonclickbait_clean.drop(['label', 'label_encoding'], axis=1).values.tolist()
nonclickbait_l_c = list(itertools.chain.from_iterable(df_nonclickbait_t))

print(nonclickbait_l_c[0:5])
print(clickbait_l_c[0:5])

['masuk radar pilwalkot medan , menantu jokowi bertemu dpw nasdem sumut', 'malaysia sudutkan ri : isu kabut asap hingga invasi babi', 'kemensos salurkan rp 7 , 3 m bagi korban kerusuhan sosial di papua', 'mpr : amandemen uud 1945 tak akan melebar ke mana-mana', 'peringati tahun baru islam , banyuwangi kembali gelar festival muharam']
['viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda', 'ada motor nyangkut di atas bambu di sleman , kok bisa ?', 'pesan gamblang poyuono menolak revisi uu kpk', 'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur', 'viral video diduga baku tembak di sleman , ini kata polisi']


# Feature Extraction: Ngram [100% DONE]

In [23]:
#these are methods that will become useful when extracting attribute markers
#why do we need all this? well... that's like 5 hours of debugging...
def flatten(foo):
    return list(_flatten(foo))

def _flatten(foo):
    for x in foo:
        if isinstance(x, collections.Iterable) and not isinstance(x, str):
            for y in _flatten(x):
                yield y
        else:
            yield x

def array_to_string(a):
    return ' '.join(flatten(a))

def is_in_string_array(elements, original): #deprecated, does not take into account sequence order
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()
# no usage in the func
def insert_string(string, inserted_string, index):
    return string[:index] + inserted_string + string[index:]

# modified from https://stackoverflow.com/questions/41752946/replacing-a-character-from-a-certain-index
def replace_string(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string. index:" + index)

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + len(newstring):]

In [24]:
# creating function for making ngram from list of headline
def ngram_maker_v2(list_sentences, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    res_ngram = {length : [] for length in lengths}

    for leng in lengths:
        for sentence in list_sentences:
            temp_set = set()
            n_grams = ngrams(nltk.word_tokenize(sentence), leng)

            for grams in n_grams:
                res = ' '.join(grams)
                temp_set.add(res)

            for key in temp_set:
                split_t = tuple(key.split(" "))
                res_ngram[leng].append(split_t)

    return res_ngram

In [25]:
# create a function that count the ngram occurance
def counter_for_ngram(ngram_dict, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    ngram_count_res = {length : [] for length in lengths}

    for key in ngram_dict:
        ng_count = Counter(ngram_dict[key])
        ngram_count_res[key] = ng_count

    return ngram_count_res

In [26]:
param_span_t = 3
clickbait_ngrams_v2= ngram_maker_v2(list_sentences=clickbait_l_c, min_length=1, max_length=param_span_t)
clickbait_ngram_count_v2 = counter_for_ngram(ngram_dict=clickbait_ngrams_v2,min_length=1, max_length=param_span_t)

non_clickbait_ngrams_v2= ngram_maker_v2(list_sentences=nonclickbait_l_c, min_length=1, max_length=param_span_t)
non_clickbait_ngram_count_v2 = counter_for_ngram(ngram_dict=non_clickbait_ngrams_v2,min_length=1, max_length=param_span_t)

In [27]:
# creat function for get ngram from the word in the one sentence reference only
def ngram_from_sentence_v3(ori_sentence, min_length, max_length):
    sent_in_arr = [ori_sentence]
    res_ngram_sentence = []
    stc_ngram_only = ngram_maker_v2(list_sentences=sent_in_arr, min_length=min_length, max_length=max_length)
    return stc_ngram_only

def count_ngram_sentence_onlyV3(ngram_dictionary, ngram_from_sentence, min_length, max_length):
    lengths = range(min_length, max_length + 1)
    ngram_compare_arr = []

    for length in lengths:
        for split in ngram_from_sentence[length]:
            ngram_compare_arr.append([split, ngram_dictionary[length][split]])

    return np.array(ngram_compare_arr)

In [28]:
# create function for get the attribute of the sentence

smoothing_parameter = 1

def array_to_string(a):
    return ' '.join(flatten(a))

# the word is in the final array;
def is_in_string_array_v2(elements, original):
    return np.isin(array_to_string(elements).split(), array_to_string(original).split()).any()

# the the attribute based on the sentence, with min_length, max_length and the reference clickbait/nonclickbait dictinoary
def get_attribute_sentence(sentence, min_length, max_length, ref_bait, param_threshold):
    # ngram sentence as reference
    sentence_ngram_ref = ngram_from_sentence_v3(ori_sentence=sentence, min_length=min_length, max_length=max_length)

    # count the clickbait sentence, clickbait ngram count as based reference
    counts_sentence_clickbait = count_ngram_sentence_onlyV3(clickbait_ngram_count_v2,sentence_ngram_ref,min_length,max_length)

    # print(counts_sentence_clickbait)

    # count the non-clickbait sentence, non-clickbait ngram count as based reference
    counts_sentence_nonclickbait = count_ngram_sentence_onlyV3(non_clickbait_ngram_count_v2,sentence_ngram_ref,min_length,max_length)

    importances = 0

    csn_int = counts_sentence_nonclickbait[:,1]
    csn_ngram = counts_sentence_nonclickbait[:,0]

    csc_int = counts_sentence_clickbait[:,1]
    csc_ngram = counts_sentence_clickbait[:,0]

    if ref_bait == "to-clickbait":
        top = (csc_int + smoothing_parameter)
        bottom = (csn_int + smoothing_parameter)
        importances = (top/bottom)
    elif ref_bait == "to-non-clickbait":
        top = (csn_int+ smoothing_parameter)
        bottom = (csc_int + smoothing_parameter)
        importances = (top/bottom)

    # print(counts_sentence_clickbait)

    de = counts_sentence_clickbait[:, 0]
    # print("impot: ", importances)

    # print(" de: ", de)
    importances_t = np.vstack((importances, de)).T

    # print("importances_t", importances_t)

    attribute_words = []
    for importance in importances_t:
        if importance[0] > param_threshold and not is_in_string_array_v2(importance[1], attribute_words):
            # print(importance[1])
            attribute_words.append(' '.join(importance[1]))

    return attribute_words

In [29]:
# run the get attribute sentence
test11 = get_attribute_sentence("viral ini ! bj habibie meninggal", 1, 2, "to-clickbait", 4 )

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_44240/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)
/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_44240/1295762271.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(x, collections.Iterable) and not isinstance(x, str):


In [30]:
def separate(sentence, style_src, param_threshold):
    attributes = get_attribute_sentence(sentence, 1, 2, "to-clickbait", param_threshold )
    c = sentence

    replace_indexes = []

    for a in attributes:
        replace_index = -1
        replace_index = c.find(a)
        replace_indexes.append(replace_index)
        c = c.replace(a, " "*len(a))

    if len(attributes) == 0:
        return {'c': c, 'a': [], 'i': [], 's': sentence}

    replace_indexes, attributes = zip(*sorted(zip(replace_indexes, attributes)))
    return {'c': c, 'a': attributes, 'i': replace_indexes, 's': sentence}

def get_c(sentence, style, param_threshold):
    return re.sub(' +', ' ', separate(sentence, style, param_threshold)['c'])

def get_a(sentence, style, param_threshold):
    a = separate(sentence, style, param_threshold)['a']
    if len(a) > 0:
        return ' '.join(a)
    else:
        return ""

In [31]:
# stc_test_retrieve = "kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur"
# get_attribute_sentence(stc_test_retrieve, 1, 2, "to-clickbait", 5)
# separate(stc_test_retrieve, "to-clickbait", 5)
#
# "Viral !"
#
# Viral
# !
# Viral !
#
# ["Viral", "!"]

# Preprocessed : TFIDF, Retrieve Function [100% DONE]

to get similar sentece in the clickbait headline list

In [32]:
# 3 of all headline list
# d_both
# nonclickbait_l_c
# clickbait_l_c

In [33]:
# import all needed librarty
tfidf_vectorizer=TfidfVectorizer(use_idf=True, stop_words=None)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(d_both)
dict_idf = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

/Users/nurrizkyimani/.conda/envs/workspace/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [34]:
param_backoff_limit = 3
def get_overlap(a, b):
    #     print(a, b)
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_weighted_overlap(a, b):
    overlap = get_overlap(a, b)
    a_counter = collections.Counter(a.split())
    #calculate
    weighted_overlap = 0
    for word in overlap:

        word_tf = a_counter[word]#/len(a.split()) -> commented out cause division by constant value doesn't matter

        get_idf = dict_idf.get(word)
        word_idf = 1 if get_idf == None else get_idf #get rid of error when idf not in dict

        word_tfidf = word_tf*word_idf
        weighted_overlap+=overlap[word]*word_tfidf

    return weighted_overlap

# 0 = nonclickbait_l_c
# 1 = clickbait_l_c

def get_similar_sentence_v2(sentence, style_src):
    opposite_dataset = clickbait_l_c if style_src else nonclickbait_l_c

    closest_sentence = ""
    highest_overlap = 0
    previous_sentences = []

    for sentence_b in opposite_dataset:
        weighted_overlap = get_weighted_overlap(sentence, sentence_b)
        if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
            highest_overlap = weighted_overlap
            closest_sentence = sentence_b

    return closest_sentence

def get_similar_sentence(sentence, style_src):

    opposite_dataset = clickbait_l_c if style_src else nonclickbait_l_c
    highest_overlap = 0
    closest_sentence = ""
    num_markers = 0
    previous_sentences = []
    backoff_count = 0

    min_attribute_markers=len(get_attribute_sentence(sentence, 1, 2, "to-non-clickbait", 5 ))

    while(num_markers < min_attribute_markers and backoff_count < param_backoff_limit):
        for sentence_b in opposite_dataset:
            weighted_overlap = get_weighted_overlap(sentence, sentence_b)
            if weighted_overlap > highest_overlap and sentence_b not in previous_sentences:
                highest_overlap = weighted_overlap
                closest_sentence = sentence_b
        highest_overlap = 0
        backoff_count += 1
        previous_sentences.append(closest_sentence)
        num_markers = len((get_attribute_sentence(closest_sentence, 1, 2, "to-clickbait", 5 )))

    return closest_sentence

In [35]:
# Retrieve using tfidf
def retrieve_v2(sentence, style_ret_src):
    style_src = 0
    if(style_ret_src == "to-clickbait"):
        style_src = 1

    similar_stc =  get_similar_sentence_v2(sentence=sentence, style_src=style_src)
    print("v2 stc: ", similar_stc)
    return separate(sentence=similar_stc, style_src= "to-clickbait", param_threshold=5)

# Retrieve using tfidf
def retrieve(sentence, style_ret_src):
    style_src = 0
    if(style_ret_src == "to-clickbait"):
        style_src = 1

    similar_stc =  get_similar_sentence(sentence=sentence, style_src=style_src)
    print("v1 stc: ", similar_stc)
    return separate(sentence=similar_stc, style_src= "to-clickbait", param_threshold=5)

In [36]:
stc_test_retrieve = "tim jibom disposal bahan peledak di rumah terduga teroris di jakut"
retrieve(stc_test_retrieve, "to-clickbait")
# retrieve_v2(stc_test_retrieve,  "to-clickbait")

v1 stc:  duarr ! bahan peledak terduga teroris jakut didisposal , warga tepuk tangan


/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_44240/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


{'c': 'duarr   bahan peledak terduga teroris jakut didisposal , warga tepuk tangan',
 'a': ('!',),
 'i': (6,),
 's': 'duarr ! bahan peledak terduga teroris jakut didisposal , warga tepuk tangan'}

In [37]:
# ref_word = "jokowi ke jakarta"
# retrive_setence = retrieve(ref_word) -> clickbait_dataset
#
# retrive_setence = "viral !  jokowi ke australia naik pesawat" -> clickbait_datset
#
#
# retrive_setence, ref_word
#
# retrive_setence => attribute "viral, ! ", content : "jokowi ke australia naik pesawat" , sentence :"viral !  jokowi ke australia naik pesawat"
#
#


# Feature Extraction: Generate, Dictionary Preprocess [100%]

In [38]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [39]:
# count the overlap; we need function for this;
def get_overlap(a, b):
    a_counter = collections.Counter(a.split())
    b_counter = collections.Counter(b.split())
    overlap = a_counter & b_counter
    return overlap

def get_total_overlap(a, b):
    return len(list(get_overlap(a, b).elements()))

In [40]:

noise_chance = 0.4
param_t = 4

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")

    d_pos_a = []
    pairs_pos = []

    for sentence in nonclickbait_l_c:
        c = get_c(sentence, 1, param_t)
        a = get_a(sentence, 1, param_t)
        d_pos_a.append(a)
        pairs_pos.append([c, a, sentence])

    d_neg_a = []
    pairs_neg = []

    for sentence in clickbait_l_c:
        c = get_c(sentence, 0, param_t)
        a = get_a(sentence, 0, param_t)
        d_neg_a.append(a)
        pairs_neg.append([c, a, sentence])

    #adding noise for pos
    for pair in pairs_pos:
        if random.random() < noise_chance:
            real_a = pair[1].split()

            if(len(real_a) == 0):
                continue

            for a in d_pos_a:
                if(len(a) == 0):
                    continue
                a = a.split()
                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    #adding noise for neg
    for pair in pairs_neg:
        if random.random() < noise_chance:
            real_a = pair[1].split()
            if(len(real_a) == 0):
                continue

            for a in d_neg_a:
                if(len(a) == 0):
                    continue
                a = a.split()

                overlap = get_total_overlap(' '.join(real_a), ' '.join(a))
                if overlap > 0 and ((overlap == len(real_a) - 1 and len(real_a) - len(a) == 1) or (overlap == len(real_a) and len(real_a) - len(a) == -1)):
                    real_a = a
                    break;
            pair[1] = ' '.join(real_a)

    pairs = np.concatenate((pairs_pos, pairs_neg), 0)
    for pair in pairs:
        input_lang.addSentence(pair[0])
        input_lang.addSentence(pair[1])
        output_lang.addSentence(pair[2])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, pairs_neg, pairs_pos

input_lang, output_lang, pairs, pairs_click, pairs_nonclick = prepareData()

pairs_click

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_44240/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


Counted words:
input 13917
output 13896


[[' driver di bekasi antar pesanan makanan sepeda',
  'viral ! ojol pakai',
  'viral ! driver ojol di bekasi antar pesanan makanan pakai sepeda'],
 ['ada motor nyangkut di atas bambu di sleman , bisa ',
  'kok ?',
  'ada motor nyangkut di atas bambu di sleman , kok bisa ?'],
 ['pesan gamblang poyuono menolak revisi uu kpk',
  '',
  'pesan gamblang poyuono menolak revisi uu kpk'],
 [' maling di rumah mewah jakut cctv bingung cari jalan ',
  'kocak ! terekam kabur',
  'kocak ! maling di rumah mewah jakut terekam cctv bingung cari jalan kabur'],
 [' diduga baku tembak di sleman , polisi',
  'viral video ini kata',
  'viral video diduga baku tembak di sleman , ini kata polisi'],
 ['waspada ada penipuan catut pertamina , korbannya rugi puluhan juta',
  'akhirnya !',
  'waspada ! ada penipuan catut pertamina , korbannya rugi puluhan juta'],
 ['jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ',
  'ini rinciannya',
  'jaksa beberkan senpi-peluru tajam pembelian kivlan zen cs , ini 

In [41]:
pc = pd.DataFrame(pairs_click)
pc

,0,1,2
0,driver di bekasi antar pesanan makanan sepeda,viral ! ojol pakai,viral ! driver ojol di bekasi antar pesanan ma...
1,"ada motor nyangkut di atas bambu di sleman , b...",kok ?,"ada motor nyangkut di atas bambu di sleman , k..."
2,pesan gamblang poyuono menolak revisi uu kpk,,pesan gamblang poyuono menolak revisi uu kpk
3,maling di rumah mewah jakut cctv bingung cari...,kocak ! terekam kabur,kocak ! maling di rumah mewah jakut terekam cc...
4,"diduga baku tembak di sleman , polisi",viral video ini kata,"viral video diduga baku tembak di sleman , ini..."
...,...,...,...
3311,ungkap perasaan karena gelandangan bakal dike...,tamara bleszynski miris ramai dipuji,tamara bleszynski ungkap perasaan miris karena...
3312,istana bantah jokowi tak peka pada korban kabu...,usai ' bareng,istana bantah jokowi tak peka pada korban kabu...
3313,"rilis teaser ' ' jelang , konsep dan koreogra...",twice mv feel special comeback,twice rilis teaser mv ' feel special ' jelang ...
3314,ada niat tambah momongan tanpa ikut program ha...,aja,ada niat tambah momongan tanpa ikut program ha...


In [42]:
pnc = pd.DataFrame(pairs_nonclick)
pnc

,0,1,2
0,"masuk radar pilwalkot medan , menantu jokowi b...",,"masuk radar pilwalkot medan , menantu jokowi b..."
1,malaysia sudutkan ri : isu kabut asap hingga i...,,malaysia sudutkan ri : isu kabut asap hingga i...
2,"kemensos salurkan rp 7 , 3 m bagi korban kerus...",,"kemensos salurkan rp 7 , 3 m bagi korban kerus..."
3,mpr : amandemen uud 1945 tak akan melebar ke m...,,mpr : amandemen uud 1945 tak akan melebar ke m...
4,"peringati tahun baru islam , banyuwangi kembal...",,"peringati tahun baru islam , banyuwangi kembal..."
...,...,...,...
5292,"wamena papua kembali membara , kantor bupati d...",,"wamena papua kembali membara , kantor bupati d..."
5293,bmkg angkat bicara soal langit merah jambi,,bmkg angkat bicara soal langit merah jambi
5294,operasional bandara wamena dihentikan akibat r...,,operasional bandara wamena dihentikan akibat r...
5295,"asap karhutla riau mulai merambah ke nias , bm...",,"asap karhutla riau mulai merambah ke nias , bm..."


In [43]:
for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
for pair in pairs:
    input_lang.addSentence(pair[1])
    output_lang.addSentence(pair[0])

print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 13917
output 13930


# Dataset:  Train, Validation, Test Split [100% Done]

In [1]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [36]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)
# train_data, test_data = train_test_split(pairs, test_size=0.3, random_state=42)
train_data, test_data  = train_test_split(pairs, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=0.18, random_state=42)

In [37]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")
print(f"Number of validation examples: {len(validation_data)}")

Number of training examples: 5649
Number of testing examples: 1723
Number of validation examples: 1241


In [38]:
print(f"{train_data[0]}")
print(test_data[0])
print(validation_data[0])

['tak perlu ke lombok-bali , gili di jawa timur yang memikat' 'inilah'
 'tak perlu ke lombok-bali , inilah gili di jawa timur yang memikat']
['hasil liga europa : eintracht frankfurt vs arsenal 0-3' ''
 'hasil liga europa : eintracht frankfurt vs arsenal 0-3']
['hotel riau akan tampung warga terkena dampak kebakaran hutan' ''
 'hotel riau akan tampung warga terkena dampak kebakaran hutan']



# STAGE 3: NN Model : Encoder, Decoder, Seq2Seq [100% Done]

In [133]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [178]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size, weight):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(input_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [179]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size, weight):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(output_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# STAGE 5: Improvement : Add fast text model

In [65]:
# !pip install --upgrade gensim

In [66]:
from gensim.models.fasttext import load_facebook_model

ft = load_facebook_model('fastext_model_tweet.bin')

In [67]:
# fast_text_path = "2022_thesis_styletransfer/cp_fastext_model_tweet.bin"
# ft = load_facebook_model(fast_text_path)

In [68]:
ft.vector_size

100

In [69]:
embedding_matrix_ft_input = np.random.random((input_lang.n_words , ft.vector_size))
embedding_matrix_ft_output = np.random.random((output_lang.n_words, ft.vector_size))

embedding_matrix_ft_input_test = np.random.random((input_lang.n_words , ft.vector_size))

# torch.zeros(max_length, encoder_a.hidden_size, device=device)

In [70]:
print(input_lang.n_words)
print(ft.vector_size)

13927
100


In [71]:
for i, word in input_lang.index2word.items():
    try:
        embedding_matrix_ft_input[i] = ft.wv[word]
    except:
        pass

for i, word in output_lang.index2word.items():
    try:
        embedding_matrix_ft_output[i] = ft.wv[word]
    except:
        pass

In [72]:
embedding_matrix_ft_input = torch.from_numpy(embedding_matrix_ft_input)
embedding_matrix_ft_output = torch.from_numpy(embedding_matrix_ft_output)

In [73]:
embedding_matrix_ft_output = embedding_matrix_ft_output.float()
embedding_matrix_ft_input = embedding_matrix_ft_input.float()

print(embedding_matrix_ft_output)

tensor([[-7.4259e-02,  1.6886e-01,  1.8216e-01,  ...,  9.6468e-02,
          2.2153e-01, -5.3748e-02],
        [-3.0592e-03, -1.5431e-03, -2.3437e-04,  ...,  1.4830e-03,
          2.1542e-03, -1.5045e-03],
        [ 9.2200e-02,  3.7329e-01, -5.2906e-01,  ..., -2.1978e-01,
         -3.4488e-01, -5.9725e-01],
        ...,
        [ 1.1046e-01,  9.3198e-02, -4.3081e-01,  ...,  7.1032e-01,
          3.3464e-02, -1.4703e-01],
        [-1.0751e-02, -1.2092e+00,  4.2839e-02,  ...,  6.0887e-02,
         -1.8061e-01, -4.9997e-01],
        [ 3.7626e-02,  2.1549e-01,  7.7507e-02,  ...,  9.6235e-02,
          1.1772e-03, -2.0547e-01]])


# STAGE 3: Word to Index Conversion

In [74]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    # print("Indexes", indexes )
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_c_tensor = tensorFromSentence(input_lang, pair[0])
    input_a_tensor = tensorFromSentence(input_lang, pair[1])
    target_tensor = tensorFromSentence(output_lang, pair[2])
    # print(input_c_tensor)
    # print(input_a_tensor)
    return (input_c_tensor, input_a_tensor, target_tensor)

In [75]:
training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(1)]
training_pairs

[(tensor([[5719],
          [5720],
          [ 876],
          [  58],
          [ 287],
          [5721],
          [4482],
          [ 133],
          [5722],
          [   4],
          [  26],
          [ 789],
          [   2],
          [   1]]),
  tensor([[13561],
          [13390],
          [13383],
          [ 1223],
          [   16],
          [13414],
          [13429],
          [13385],
          [    1]]),
  tensor([[6151],
          [6152],
          [1017],
          [  71],
          [1240],
          [  45],
          [ 325],
          [6153],
          [  11],
          [4892],
          [ 161],
          [6154],
          [   6],
          [  32],
          [1444],
          [  19],
          [ 306],
          [ 443],
          [ 922],
          [  22],
          [   1]]))]

In [76]:
#Training
MAX_LENGTH = 50

teacher_forcing_ratio = 0.5


def train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a, decoder, encoder_c_optimizer, encoder_a_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    # print("encoder_a_hidden", encoder_a_hidden)

    encoder_a_optimizer.zero_grad()
    encoder_c_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_c_length = input_c_tensor.size(0)
    input_a_length = input_a_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    loss = 0

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(
            input_c_tensor[ei], encoder_c_hidden)
        encoder_c_outputs[ei] = encoder_c_output[0, 0]

    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(
            input_a_tensor[ei], encoder_a_hidden)
        encoder_a_outputs[ei] = encoder_a_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_c_optimizer.step()
    encoder_a_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

# STAGE 3: NN Model : Training Architecture

In [77]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [258]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [259]:
def trainIters(
        encoder_a,
        encoder_c,
        decoder,
        n_iters,

        criterion,
        encoder_c_optimizer,
        encoder_a_optimizer,
        decoder_optimizer,

        print_every=1000,
        plot_every=100,
        learning_rate=0.01):

    encoder_a.train()
    encoder_c.train()
    decoder.train()

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
    # encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
    # decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    # criterion = nn.NLLLoss()

    ouput_loss_total = 0

    for iter in range(1, n_iters + 1):

        training_pair = training_pairs[iter - 1]
        input_a_tensor = training_pair[0]
        input_c_tensor = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_c_tensor, input_a_tensor, target_tensor,
                     encoder_c, encoder_a, decoder,
                     encoder_c_optimizer, encoder_c_optimizer, decoder_optimizer,
                     criterion)

        print_loss_total += loss
        plot_loss_total += loss
        ouput_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

    return ouput_loss_total/n_iters

In [260]:
def evaluating(
        encoder_a,
        encoder_c,
        decoder,

        criterion,
        n_iters,
        plot_every : int =10,
        max_length=MAX_LENGTH
):

    encoder_a.eval()
    encoder_c.eval()
    decoder.eval()

    plot_losses = []
    plot_loss_total = 0
    loss_total_output = 0

    training_pairs = [tensorsFromPair(random.choice(pairs_click)) for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    start: float = time.time()

    with torch.no_grad():
        for iter  in  range(1, n_iters + 1):
            # taking the pair and extract the c, a, target
            training_pair = training_pairs[iter - 1]
            input_c_tensor = training_pair[0]
            input_a_tensor = training_pair[1]
            target_tensor = training_pair[2]

            # START: eval func
            # init hidden
            encoder_c_hidden = encoder_c.initHidden()
            encoder_a_hidden = encoder_a.initHidden()

            # len of sentence of c, a, t, that already; e.g input_c_length= len(input_c_tensor)
            input_c_length = input_c_tensor.size(0)
            input_a_length = input_a_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
            encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

            loss = 0

            for ei in range(input_c_length):
                encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei], encoder_c_hidden)
                encoder_c_outputs[ei] = encoder_c_output[0, 0]

            for ei in range(input_a_length):
                encoder_a_output, encoder_a_hidden = encoder_a(input_a_tensor[ei], encoder_a_hidden)
                encoder_a_outputs[ei] = encoder_a_output[0, 0]

            decoder_input = torch.tensor([[SOS_token]], device=device)
            decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    loss += criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

            else:
            # Without teacher forcing: use its own predictions as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break

            l = loss.item() / target_length
            # END: Eval Func

            loss_total_output += l
            plot_loss_total += l

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                # print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                #                              iter, iter / n_iters * 100, plot_loss_avg))
                plot_loss_total = 0

        showPlot(plot_losses)

    return loss_total_output/ n_iters

In [261]:
# word_vec_size = 128
word_vec_size = ft.vector_size
hidden_size = 512

# encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words).to(device)

encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)

decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

In [262]:
# trainIters(encoder_c, encoder_a, decoder, 200, print_every=100)
# len(pairs_click)
print(len(pairs) - len(pairs_click))
print(len(pairs_click))
print(len(pairs))

5297
3316
8613


In [263]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [268]:
N_EPOCHS = 1
best_valid_loss = float('inf')

learning_rate=0.01

encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)

TRAINING_DATASET_SIZE = 2000
EVALUATION_DATASET_SIZE = 500

criterion = nn.NLLLoss()

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder_a,
        encoder_c,
        decoder,
        TRAINING_DATASET_SIZE,

        criterion,
        encoder_c_optimizer,
        encoder_a_optimizer,
        decoder_optimizer,
        print_every=50
    )

    valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=50)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder_a.state_dict(), 'encoder_a-model.pt')
        torch.save(encoder_c.state_dict(), 'encoder_c-model.pt')
        torch.save(decoder.state_dict(), 'decoder-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

0m 34s (- 22m 43s) (50 2%) 6.8913
1m 6s (- 21m 3s) (100 5%) 6.8099
1m 37s (- 20m 5s) (150 7%) 7.1184
2m 7s (- 19m 4s) (200 10%) 6.9466
2m 36s (- 18m 18s) (250 12%) 6.9606
3m 12s (- 18m 8s) (300 15%) 7.0870
3m 44s (- 17m 37s) (350 17%) 6.9832
4m 16s (- 17m 4s) (400 20%) 6.9154
4m 57s (- 17m 3s) (450 22%) 7.3593
5m 28s (- 16m 26s) (500 25%) 7.1346
5m 58s (- 15m 43s) (550 27%) 6.6985
6m 27s (- 15m 3s) (600 30%) 6.9822
7m 1s (- 14m 35s) (650 32%) 7.3053


KeyboardInterrupt: 

# Stage 3: Testing the DeleteRetrieve to Generate Text

In [ ]:
def encoderStep(encoder_c, encoder_a, c, a, max_length=MAX_LENGTH):
    input_c_tensor = tensorFromSentence(input_lang, c)
    input_a_tensor = tensorFromSentence(input_lang, a)

    input_c_length = input_c_tensor.size()[0]
    input_a_length = input_a_tensor.size()[0]

    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei],
                                                       encoder_c_hidden)
        encoder_c_outputs[ei] += encoder_c_output[0, 0]


    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(input_a_tensor[ei],
                                                       encoder_a_hidden)
        encoder_a_outputs[ei] += encoder_a_output[0, 0]



    return torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

In [ ]:
def get_c_embedding(encoder_c, c, max_length=MAX_LENGTH):
    input_c_tensor = tensorFromSentence(input_lang, c)

    input_c_length = input_c_tensor.size()[0]

    encoder_c_hidden = encoder_c.initHidden()

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(input_c_tensor[ei],
                                                       encoder_c_hidden)
        encoder_c_outputs[ei] += encoder_c_output[0, 0]

    return encoder_c_hidden

In [ ]:
ALPHA = 0.7

class Node:
    def __init__(self, p_t, i, d_hidden, lvl, node_parent=None):
        self.p_t = p_t
        self.i = i
        self.d_hidden = d_hidden
        self.lvl = lvl
        self.node_parent = node_parent

        self.p_sentence = self.sentenceProb()

    def prepareToDecode(self):
        d_in = self.i.squeeze()#.detatch()
        return d_in, self.d_hidden

    def sentenceProb(self):
        if self.node_parent == None:
            return self.p_t
        return self.node_parent.sentenceProb() + self.p_t #addition because of log

    def normProb(self):
        #Normalised probability
        #         return 1/((self.lvl+1)**ALPHA) * self.sentenceProb()
        #perplexity
        return torch.exp(self.sentenceProb()) ** (-1/(self.lvl+1))

    def getToken(self):
        return output_lang.index2word[self.i.item()]

    def getTokens(self):
        token = [self.getToken()]

        if(self.node_parent == None):
            return token
        else:
            return self.node_parent.getTokens() + token

    def getSentence(self):
        return ' '.join(self.getTokens())

In [ ]:
BEAM_WIDTH = 10

def evaluate(encoder_c, encoder_a, decoder, c, a, max_length=MAX_LENGTH):
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep(encoder_c, encoder_a, c, a)
        decoded_words = []

        #BEAM SEARCH

        nodes = []
        finished_nodes = []
        for di in range(max_length):
            if di == 0:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                top_ps = top_ps.view(-1)
                top_is = top_is.view(-1)
                for index in range(len(top_is)):
                    p_t = top_ps[index]
                    i = top_is[index]
                    nodes.append(Node(p_t, i, decoder_hidden, di))
            else:
                prev_nodes = [x for x in nodes if x.lvl == di-1]
                prev_nodes = sorted(prev_nodes, key=lambda x: x.p_sentence.item(), reverse=True)
                prev_nodes = prev_nodes[:BEAM_WIDTH]
                nodes = []
                for node in prev_nodes:
                    decoder_output, decoder_hidden = decoder(*node.prepareToDecode())
                    top_ps, top_is = decoder_output.data.topk(BEAM_WIDTH)
                    top_ps = top_ps.view(-1)
                    top_is = top_is.view(-1)

                    for index in range(len(top_is)):
                        p_t = top_ps[index]
                        i = top_is[index]
                        child_node = Node(p_t, i, decoder_hidden, di, node)

                        if i.item() == EOS_token:
                            finished_nodes.append(child_node)
                        else:
                            nodes.append(child_node)

        final_node = sorted(finished_nodes, key=lambda x: x.p_sentence.item(), reverse=True)[0]
        return final_node.getTokens()

# evaluate(encoder_c, encoder_a, decoder, "the food", "great")

In [ ]:
def evaluateWithoutBeamSearch(encoder_c, encoder_a, decoder, c, a, max_length=MAX_LENGTH):
    with torch.no_grad():
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoderStep(encoder_c, encoder_a, c, a)
        decoded_words = []

        #BEAM SEARCH
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)

            topv, topi = F.softmax(decoder_output).data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words
# ' '.join(evaluateWithoutBeamSearch(encoder_c, encoder_a, decoder, "the food is", "great"))

In [ ]:
def evaluateRandomly(encoder_c, encoder_a, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder_c, encoder_a, decoder, pair[0], pair[1])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
evaluateRandomly(encoder_c, encoder_a, decoder)

In [ ]:
def DeleteAndRetrieve(sentence, style):
    c_src = separate(sentence, 0)['c']
    a_tgt = ' '.join(retrieve(sentence, not style)['a'])
    return ' '.join(evaluate(encoder_c, encoder_a, decoder, c_src, a_tgt))

# STAGE 4: ADDITIONAL: FULL COPY ONLY ONE EPOCH

In [ ]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, word_vec_size)
        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, word_vec_size)
        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_c_tensor = tensorFromSentence(input_lang, pair[0])
    input_a_tensor = tensorFromSentence(input_lang, pair[1])
    target_tensor = tensorFromSentence(output_lang, pair[2])
    return (input_c_tensor, input_a_tensor, target_tensor)

In [ ]:
#Training
MAX_LENGTH = 50

teacher_forcing_ratio = 0.5


def train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a, decoder, encoder_c_optimizer, encoder_a_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_c_hidden = encoder_c.initHidden()
    encoder_a_hidden = encoder_a.initHidden()

    encoder_a_optimizer.zero_grad()
    encoder_c_optimizer.zero_grad()

    decoder_optimizer.zero_grad()

    input_c_length = input_c_tensor.size(0)
    input_a_length = input_a_tensor.size(0)

    target_length = target_tensor.size(0)

    encoder_c_outputs = torch.zeros(max_length, encoder_c.hidden_size, device=device)
    encoder_a_outputs = torch.zeros(max_length, encoder_a.hidden_size, device=device)

    loss = 0

    for ei in range(input_c_length):
        encoder_c_output, encoder_c_hidden = encoder_c(
            input_c_tensor[ei], encoder_c_hidden)
        encoder_c_outputs[ei] = encoder_c_output[0, 0]

    for ei in range(input_a_length):
        encoder_a_output, encoder_a_hidden = encoder_a(
            input_a_tensor[ei], encoder_a_hidden)
        encoder_a_outputs[ei] = encoder_a_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = torch.cat((encoder_c_hidden, encoder_a_hidden), 2)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_c_optimizer.step()
    encoder_a_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder_a, encoder_c, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_c_optimizer = optim.Adadelta(encoder_c.parameters(), lr=learning_rate)
    encoder_a_optimizer = optim.Adadelta(encoder_a.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_a_tensor = training_pair[0]
        input_c_tensor = training_pair[1]

        target_tensor = training_pair[2]
        loss = train(input_c_tensor, input_a_tensor, target_tensor, encoder_c, encoder_a,
                     decoder, encoder_c_optimizer, encoder_c_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
word_vec_size = 128
hidden_size = 512
encoder_c = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
encoder_a = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)

decoder = DecoderRNN(hidden_size + hidden_size, word_vec_size, output_lang.n_words).to(device)

In [ ]:
# len(pairs)

In [ ]:
trainIters(encoder_c, encoder_a, decoder, 8613, print_every=200)

# Stage 5: DeleteOnly

In [78]:
# Resources: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
# Had to modify & adapt most of the code in the tutorial since this isn't translation & data preprocessing is different

SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [79]:
param_t = 4

def prepareData():
    input_lang = Lang("input")
    output_lang = Lang("output")
    pairs = []
    for sentence in clickbait_l_c:
        pairs.append([get_c(sentence, 1, param_t), sentence, 1])

    for sentence in nonclickbait_l_c:
        pairs.append([get_c(sentence, 0, param_t), sentence, 0])

    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs



In [80]:
input_lang, output_lang, pairs = prepareData()

/var/folders/1n/2gdy6g815s301ygx2s35p6_80000gp/T/ipykernel_44240/4054946038.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(ngram_compare_arr)


Counted words:
input 13380
output 13896


In [81]:
for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
for pair in pairs:
    input_lang.addSentence(pair[1])
    output_lang.addSentence(pair[0])

print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)

Counted words:
input 13927
output 13927


In [82]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, word_vec_size, hidden_size, weight):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(input_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.gru = nn.GRU(word_vec_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [83]:
class StyleEmbedder(nn.Module):
    def __init__(self, num_styles, dimensions):
        super(StyleEmbedder, self).__init__()
        self.dimensions = dimensions
        self.embedding = nn.Embedding(num_styles, dimensions)

    def forward(self, input):
        embedded = self.embedding(input).view(1, 1, -1)
        return embedded

In [84]:
class Maxout(nn.Module):
    def __init__(self, pool_size):
        super().__init__()
        self._pool_size = pool_size

    def forward(self, x):
        assert x.shape[1] % self._pool_size == 0, \
            'Wrong input last dim size ({}) for Maxout({})'.format(x.shape[1], self._pool_size)
        m, i = x.view(*x.shape[:1], x.shape[1] // self._pool_size, self._pool_size, *x.shape[2:]).max(2)
        return m

In [85]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, word_vec_size, output_size,weight):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # self.embedding = nn.Embedding(output_size, word_vec_size)
        self.embedding = nn.Embedding.from_pretrained(weight)

        self.gru = nn.GRU(word_vec_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.maxout = Maxout(1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = self.maxout(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [86]:
#Preparing Training Data
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorFromStyle(style):
    #     one_hot_encoded_style = []
    #     if style:
    #         one_hot_encoded_style = [1,0]
    #     else:
    #         one_hot_encoded_style = [0,1]
    return torch.tensor(style, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    style_tensor = tensorFromStyle(pair[2])
    return (input_tensor, style_tensor, target_tensor)  #add style_tensor

In [87]:
#Training
MAX_LENGTH = 50

teacher_forcing_ratio = 0.5


def train(
        input_tensor,
        style_tensor,
        target_tensor,

        encoder,
        style_embedder,
        decoder,

        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,

        criterion,
        max_length=MAX_LENGTH):


    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    style_embedder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]


    decoder_input = torch.tensor([[SOS_token]], device=device)

    #calculate style embedding
    style_embedding = style_embedder(style_tensor)

    decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    #style embedding
    style_embedder_optimizer.step()

    decoder_optimizer.step()


    return loss.item() / target_length

In [88]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [89]:
def trainIters(
        encoder,
        style_embedder,
        decoder,

        criterion,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,

        n_iters,
        print_every=1000,
        plot_every=100,
        learning_rate=0.01):

    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder.train()
    style_embedder.train()
    decoder.train()

    # encoder_optimizer = optim.Adadelta(encoder.parameters(), lr=learning_rate)
    # decoder_optimizer = optim.Adadelta(decoder.parameters(), lr=learning_rate)
    # style_embedder_optimizer = optim.Adadelta(style_embedder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs))for i in range(n_iters)]
    # criterion = nn.NLLLoss()

    ouput_loss_total = 0

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        style_tensor = training_pair[1]
        target_tensor = training_pair[2]

        loss = train(input_tensor, style_tensor, target_tensor, encoder, style_embedder,
                     decoder, encoder_optimizer, style_embedder_optimizer, decoder_optimizer, criterion)

        print_loss_total += loss
        plot_loss_total += loss
        ouput_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    return ouput_loss_total/n_iters

In [90]:
def evaluating(
        encoder,
        style_embedder,
        decoder,

        criterion,
        n_iters,
        plot_every : int =10,
        max_length=MAX_LENGTH
):

    encoder.eval()
    style_embedder.eval()
    decoder.eval()

    plot_losses = []
    plot_loss_total = 0
    loss_total_output = 0

    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    # training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]

    start: float = time.time()

    with torch.no_grad():
        for iter  in  range(1, n_iters + 1):
            # taking the pair and extract the c, a, target
            training_pair = training_pairs[iter - 1]
            input_tensor = training_pair[0]
            style_tensor = training_pair[1]
            target_tensor = training_pair[2]

            encoder_hidden = encoder.initHidden()

            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            #style embedding
            style_embedder_optimizer.zero_grad()


            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(
                    input_tensor[ei], encoder_hidden)
                encoder_outputs[ei] = encoder_output[0, 0]


            decoder_input = torch.tensor([[SOS_token]], device=device)

            #calculate style embedding
            style_embedding = style_embedder(style_tensor)

            decoder_hidden = torch.cat((encoder_hidden, style_embedding), 2) #TODO: concatenate style embedding

            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

            if use_teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    loss += criterion(decoder_output, target_tensor[di])
                    decoder_input = target_tensor[di]  # Teacher forcing

            else:
                # Without teacher forcing: use its own predictions as the next input
                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(
                        decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    decoder_input = topi.squeeze().detach()  # detach from history as input

                    loss += criterion(decoder_output, target_tensor[di])
                    if decoder_input.item() == EOS_token:
                        break

            l = loss.item() / target_length
            # END: Eval Func

            loss_total_output += l
            plot_loss_total += l

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                # print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                #                              iter, iter / n_iters * 100, plot_loss_avg))
                plot_loss_total = 0

        showPlot(plot_losses)

    return loss_total_output/ n_iters

In [92]:
# word_vec_size = 128
word_vec_size = ft.vector_size

hidden_size = 512
style_vec_size = 128

# encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size).to(device)
# decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words).to(device)

encoder1 = EncoderRNN(input_lang.n_words, word_vec_size, hidden_size, embedding_matrix_ft_input).to(device)
decoder1 = DecoderRNN(hidden_size + style_vec_size, word_vec_size, output_lang.n_words, embedding_matrix_ft_output).to(device)

style_embedder1 = StyleEmbedder(2, style_vec_size).to(device)

In [94]:
N_EPOCHS = 10
best_valid_loss = float('inf')

learning_rate=0.1

encoder_optimizer = optim.Adadelta(encoder1.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adadelta(decoder1.parameters(), lr=learning_rate)
style_embedder_optimizer = optim.Adadelta(style_embedder1.parameters(), lr=learning_rate)

TRAINING_DATASET_SIZE = 500
EVALUATION_DATASET_SIZE = 100

criterion1 = nn.NLLLoss()

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = trainIters(
        encoder1,
        style_embedder1,
        decoder1,

        criterion1,
        encoder_optimizer,
        style_embedder_optimizer,
        decoder_optimizer,
        TRAINING_DATASET_SIZE,
        print_every=20
    )

    # valid_loss = evaluating(encoder_a, encoder_c, decoder, criterion, EVALUATION_DATASET_SIZE, plot_every=20)

    valid_loss = evaluating(
        encoder1,
        style_embedder1,
        decoder1,
        criterion1,
        EVALUATION_DATASET_SIZE,
        plot_every=20)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(encoder_a.state_dict(), 'encoder_a-model.pt')
        torch.save(encoder_c.state_dict(), 'encoder_c-model.pt')
        torch.save(decoder.state_dict(), 'decoder-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    print("Done")

0m 6s (- 2m 29s) (20 4%) 5.6562
0m 12s (- 2m 23s) (40 8%) 6.7899
0m 17s (- 2m 11s) (60 12%) 6.1643
0m 24s (- 2m 10s) (80 16%) 6.7491
0m 32s (- 2m 9s) (100 20%) 7.1132
0m 39s (- 2m 4s) (120 24%) 7.6527
0m 46s (- 2m 0s) (140 28%) 7.2228
0m 53s (- 1m 53s) (160 32%) 7.1589
0m 59s (- 1m 45s) (180 36%) 7.3757
1m 5s (- 1m 38s) (200 40%) 7.0133
1m 12s (- 1m 32s) (220 44%) 6.8911
1m 24s (- 1m 31s) (240 48%) 7.1854
1m 30s (- 1m 23s) (260 52%) 6.7948
1m 35s (- 1m 15s) (280 56%) 6.3255
1m 42s (- 1m 8s) (300 60%) 7.1648
1m 48s (- 1m 1s) (320 64%) 7.3104
1m 55s (- 0m 54s) (340 68%) 7.3726
2m 2s (- 0m 47s) (360 72%) 7.3812
2m 8s (- 0m 40s) (380 76%) 7.4629
2m 15s (- 0m 33s) (400 80%) 7.2371
2m 21s (- 0m 27s) (420 84%) 7.0916


KeyboardInterrupt: 

# Create Clickbait title for Nonclickbait dataset

In [ ]:
pairs

# BLEU Evaluation Metrics